In [1]:
"""
!pip install Augmentor
!pip install pillow
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip install keras
!unzip ./python/images_background.zip
!unzip ./python/images_background_small1.zip
!unzip ./python/images_background_small2.zip
!unzip ./python/images_evaluation.zip
"""

'\n!pip install Augmentor\n!pip install pillow\n!pip install pandas\n!pip install matplotlib\n!pip install numpy\n!pip install keras\n!unzip ./python/images_background.zip\n!unzip ./python/images_background_small1.zip\n!unzip ./python/images_background_small2.zip\n!unzip ./python/images_evaluation.zip\n'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from load_data import load_directory, train_gen, val_gen, quiz_models, LossTracker, image_from_index
from build_models import make_convnet, make_capsnet, train_convnet, train_capsnet, plot_history
from keras.models import load_model

/home/alliedtoasters/miniconda3/envs/dev1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
batch_size = 30
directory = './images_evaluation/Atemayar_Qelisayer/'
train, test, labels = load_directory(directory)
loss_tracker = LossTracker()
tg = train_gen(train, labels, batch_size=batch_size, augmentation=True, loss_obj=loss_tracker)
vg = val_gen(test, labels, batch_size=380) #Use the entire validation set here.

In [4]:
def get_train_img(idx):
    """Shortcut to display training image."""
    path = './train/'
    df = train
    img = image_from_index(idx, df, path)
    return img

In [5]:
input_shape = (105, 105, 1)
n_class = len(labels)
routings = 3
reconstruction_loss = .392

In [6]:
convnet = make_convnet(input_shape, n_class, width=16, dropout=.5)
convnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 105, 105, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 97, 97, 16)        1312      
_________________________________________________________________
conv2 (Conv2D)               (None, 93, 93, 32)        12832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 93, 93, 32)        0         
_________________________________________________________________
maxp1 (MaxPooling2D)         (None, 46, 46, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 45, 45, 32)        4128      
_________________________________________________________________
conv4 (Conv2D)               (None, 44, 44, 64)        8256      
__________

In [ ]:
history = train_convnet(convnet, tg, vg, directory, verbose=True, loss_obj=loss_tracker, epochs=100)

Epoch 1/100
 90/100 [==========================>...] - ETA: 54s - loss: 3.2607 - categorical_accuracy: 0.0344

In [ ]:
plot_history(history)

In [ ]:
results1, results2, results3 = quiz_models(directory, test, labels)

In [ ]:
from load_data import quiz_batch

def visualize_quiz(X, Y, prediction, quiz_labels):
    """Visualizes the results of a 'quiz'.
    for presentation.
    """
    quiz_Y = Y[:, quiz_labels]
    quiz_prediction = prediction[:, quiz_labels]
    correct_answers = np.argmax(quiz_Y, axis=0)
    predicted_answers = np.argmax(quiz_prediction, axis=0)
    correct_idx = np.argwhere(correct_answers == predicted_answers)[0][0]
    correct_class = quiz_labels[0]#predicted_answers[correct_idx]
    #Correct answer example
    train_img = get_train_img(correct_class)
    fig, ax = plt.subplots(5, 5, figsize=(8, 8))
    fig.suptitle('Correct Result')
    current_label = 0
    for i in range(5):
        for j in range(5):
            if i == 0:
                if j == 2:
                    ax[i, j].set_title('training image')
                    ax[i, j].imshow(train_img, cmap='Greys_r')
                    ax[i, j].set_xticks([])
                    ax[i, j].set_yticks([])
                else:
                    ax[i, j].set_visible(False)
            else:
                #likelihood = quiz_Y[current_label, correct_idx]
                lbl = np.argmax(quiz_Y[current_label, :])
                img = X[current_label, :, :, 0]
                #title = '{}'.format(str(likelihood))[:6]
                #if quiz_Y[current_label, correct_idx] == 1:
                #    title += ('(correct)')
                title = 'label {}'.format(lbl)
                ax[i, j].set_title(title)
                ax[i, j].set_xticks([])
                ax[i, j].set_yticks([])
                ax[i, j].imshow(img, cmap='Greys_r')
                current_label += 1
    plt.show()
    return
    
    

def quiz(model, df, labels, path='./eval/', verbose=1, visualize=False):
    """Runs the 'quiz' used in the paper:
    Given 20 choices, choose the character belonging
    to the correct class.
    Because this is a COMPUTER, we'll reframe the test:
    1. Choose 20 characters, one from each class.
    2. For each class label, assign the max value to that class.
    3. Return classwise accuracy.
    """
    visualized = False
    correct = 0
    total = 0
    classified = 0
    total_classified = 0
    for n in range(2, 21):
        test_no = str(n)
        if len(test_no) == 1:
            test_no = '0' + test_no
        test_samples = df[['_' + test_no in x for x in df.file]].sample(20, replace=False)
        quiz_labels = test_samples.label.values.astype(int)
        X, Y = quiz_batch(test_samples.index, df=df, labels=labels)
        quiz_Y = Y[:, quiz_labels]
        prediction = model.predict(X)
        quiz_prediction = prediction[:, quiz_labels]
        correct_answers = np.argmax(quiz_Y, axis=0)
        predicted_answers = np.argmax(quiz_prediction, axis=0)
        num_correct = np.where(correct_answers == predicted_answers, 1, 0).sum()
        if visualize and not visualized:
            if 0 < num_correct < 20:
                visualize_quiz(X, Y, prediction, quiz_labels)
                visualized = True
        correct += num_correct
        total += len(correct_answers)
        if verbose==2:
            print(
                'quiz no.', 
                test_no, 
                ' results: {}/{} correct.'.format(
                    num_correct, 
                    len(correct_answers)
                ))
        correct_classifications = np.argmax(Y, axis=1)
        classifications = np.argmax(prediction, axis=1)
        num_classified = np.where(correct_classifications == classifications, 1, 0).sum()
        classified += num_classified
        total_classified += len(correct_classifications)
        if verbose==2:
            print(
                'quiz no.', 
                test_no, 
                ' classification results: {}/{} correct.'.format(
                    num_classified, 
                    len(correct_classifications)
                ))
    if verbose==1:
        print('Quiz results: {} out of {} correct.'.format(correct, total))
        print('Quiz accuracy score: {}. Error rate: {}'.format(
            round(correct/total, 3), 
            round(1-correct/total, 3))
             )
        print('Classification results: {} out of {} correct.'.format(classified, total_classified))
        print('Classification accuracy score: {}. Error rate: {}'.format(
            round(classified/total_classified, 3), 
            round(1-classified/total_classified, 3))
             )
    return correct, total, classified, total_classified

quiz(convnet, test, labels, visualize=True)

In [ ]:
capsnet, eval_model, manipulate_model = make_capsnet(input_shape, n_class, routings, reconstruction_loss)

In [ ]:
loss_tracker = LossTracker(scale_by=3)
tg = train_gen(train, labels, batch_size=batch_size, augmentation=True, loss_obj=loss_tracker)
vg = val_gen(test, labels, batch_size=batch_size)
history = train_capsnet(capsnet, tg, vg, directory, verbose=True, loss_obj=loss_tracker)

In [ ]:
results_caps1, results_caps2, results_caps3 = quiz_models(directory, test, labels, capsnet=True)

In [ ]:
manipulate_model.set_weights(capsnet.get_weights())
eval_model.set_weights(capsnet.get_weights())

In [ ]:
test.loc[[5, 10, 12]].label

In [ ]:
def plot_training_characters(train):
    """Plots the training characters for presentation"""
    

def plot_outputs(model, val_gen, test):
    """Looks at outputs of capsnet."""
    fig, ax = plt.subplots()
    for i in range(20):
        X, Y = next(val_gen)
        Y_pred, reconstructed = model.predict(X)
        img = X[0, :, :, 0]
        plt.title('input character (label {})'.format(np.argmax(Y[0, :])));
        plt.imshow(img, cmap='Greys_r');
        plt.show();
        plt.title('reconstructed character (predicted label {})'.format(np.argmax(Y_pred[0, :])));
        plt.imshow(reconstructed[0, :, :, 0], cmap='Greys_r');
        plt.show();
    return

plot_outputs(eval_model, vg, test)

In [ ]:
eval_model.outputs

In [ ]:
np.argmax(np.array([
    [0, 0, 0],
    [1, 1, 1]
]), axis=0)